In [2]:
import openpyxl as xl
from openpyxl.styles import PatternFill, Alignment
from openpyxl.formatting.rule import CellIsRule
from openpyxl.utils import get_column_letter
from openpyxl.chart import BarChart, Reference
import pandas as pd
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go

# ---------------- Excel Processing Section ----------------
file_path = "S2PythonInternal.xlsx"   # Path to input Excel file
wb = xl.load_workbook(file_path)
sheet = wb.active

# Column indices for marks in Excel
series1_marks = 3
series2_marks = 4
assgn1_marks = 5
assgn2_marks = 6
series_avg_marks = 7
assgn_avg_marks = 8
internal_marks = 9

# Maximum marks for each evaluation component
max_marks = {
    series1_marks: 40,
    series2_marks: 40,
    assgn1_marks: 10,
    assgn2_marks: 10,
    series_avg_marks: 40,
    assgn_avg_marks: 10,
    internal_marks: 50
}

# Determine last non-empty row in the Roll No column
max_row = sheet.max_row
while max_row > 0 and sheet.cell(max_row, 1).value is None:
    max_row -= 1

# Determine last filled column in header row (row 4)
max_col = sheet.max_column
while max_col > 0 and sheet.cell(4, max_col).value is None:
    max_col -= 1

# Apply wrap text and center alignment to header row
for col in range(1, max_col + 1):
    cell = sheet.cell(4, col)
    cell.alignment = Alignment(wrap_text=True, vertical='center', horizontal='center')

# Adjust row height to accommodate wrapped header text
sheet.row_dimensions[4].height = 30

# Calculate averages and total internal marks for each student
for row in range(5, max_row + 1):
    s1 = sheet.cell(row, series1_marks).value
    s2 = sheet.cell(row, series2_marks).value
    a1 = sheet.cell(row, assgn1_marks).value
    a2 = sheet.cell(row, assgn2_marks).value

    if s1 is not None and s2 is not None:
        sheet.cell(row, series_avg_marks).value = round((s1 + s2) / 2, 0)
    if a1 is not None and a2 is not None:
        sheet.cell(row, assgn_avg_marks).value = round((a1 + a2) / 2, 0)

    sa = sheet.cell(row, series_avg_marks).value
    aa = sheet.cell(row, assgn_avg_marks).value
    if sa is not None and aa is not None:
        sheet.cell(row, internal_marks).value = sa + aa

# Highlight cells in red if marks are below 50% of maximum
red_fill = PatternFill(start_color="FF9999", end_color="FF9999", fill_type="solid")
for col, max_mark in max_marks.items():
    threshold = max_mark * 0.5
    col_letter = get_column_letter(col)
    rule = CellIsRule(operator="lessThan", formula=[str(threshold)], fill=red_fill)
    sheet.conditional_formatting.add(f"{col_letter}5:{col_letter}{max_row}", rule)

# Create a bar chart in Excel for internal marks distribution
chart = BarChart()
data = Reference(sheet, min_col=internal_marks, min_row=4, max_row=max_row)
cats = Reference(sheet, min_col=2, min_row=5, max_row=max_row)  # Student roll numbers
chart.add_data(data, titles_from_data=True)
chart.set_categories(cats)
chart.title = "Internal Marks Distribution"
sheet.add_chart(chart, f"A{max_row + 2}")  # Place chart below data

# Save the updated Excel workbook
wb.save("S2PythonInternal_updated.xlsx")

# ---------------- Pandas + Plotly Analysis Section ----------------
# Load Excel data into Pandas for interactive visualization
file_path = "S2PythonInternal_updated.xlsx"
df = pd.read_excel(file_path, skiprows=3)
df = df.iloc[:, :9]

# Rename columns for clarity
df.columns = [
    "Roll No", "Name",
    "Series 1", "Series 2",
    "Assignment 1", "Assignment 2",
    "Series Avg", "Assignment Avg",
    "Internal"
]

# Compute difference between series and assignment scores
df["Series Diff"] = df["Series 2"] - df["Series 1"]
df["Assignment Diff"] = df["Assignment 2"] - df["Assignment 1"]

# ---------------- Interactive Visualization ----------------
# Create subplot layout: row 1 for internal marks, row 2 for differences
fig = sp.make_subplots(
    rows=2, cols=1,
    subplot_titles=("Internal Marks Distribution", "Series vs Assignment Differences")
)

# Bar chart: Internal marks for each student
fig.add_trace(
    go.Bar(
        x=df["Roll No"],
        y=df["Internal"],
        text=df["Name"],
        hovertemplate=(
            "Name: %{text}<br>"
            "Internal: %{y}<br>"
            "Series Avg: %{customdata[0]}<br>"
            "Assignment Avg: %{customdata[1]}<br>"
            "Series Diff: %{customdata[2]}<br>"
            "Assignment Diff: %{customdata[3]}"
        ),
        customdata=df[["Series Avg", "Assignment Avg", "Series Diff", "Assignment Diff"]],
        marker=dict(color=df["Internal"], colorscale="Viridis"),
        name="Internal"
    ),
    row=1, col=1
)

# Bar charts: Differences in series and assignment scores
fig.add_trace(
    go.Bar(
        x=df["Roll No"],
        y=df["Series Diff"],
        name="Series 2 - Series 1",
        marker_color="indianred"
    ),
    row=2, col=1
)
fig.add_trace(
    go.Bar(
        x=df["Roll No"],
        y=df["Assignment Diff"],
        name="Assignment 2 - Assignment 1",
        marker_color="royalblue"
    ),
    row=2, col=1
)

# Update layout for better readability
fig.update_layout(
    height=800,
    showlegend=True,
    title_text="Student Performance Analysis"
)

fig.show()
